<a href="https://colab.research.google.com/github/orlovaea/nlp_ha/blob/main/nlp_ha12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HA12. Reinforcement learning from human feedback

In [ ]:
 %pip install transformers trl wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB

In [ ]:
%pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

In [ ]:
config = PPOConfig(
    model_name="ai-forever/rugpt3small_based_on_gpt2",
    learning_rate=1.41e-5,
    log_with=None,
    mini_batch_size=16
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = load_dataset("csv", data_files='drive/MyDrive/labeled.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9ed4b8ba9195d5bc/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def build_dataset(config,
                  dataset_name=dataset['train'],
                  input_min_text_length=2,
                  input_max_text_length=8):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = dataset
    ds = ds.rename_columns({"comment": "review"})

    # еще есть фильтрация по длине
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)
    ds = ds.filter(lambda x: len(x["review"]) < 2000, batched=False)

    # длина кусочка определяется случайно
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [ ]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Filter:   0%|          | 0/14412 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3511 [00:00<?, ? examples/s]

Map:   0%|          | 0/3454 [00:00<?, ? examples/s]

In [ ]:
# active_model
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
# reference_model (обратите внимание что это одна и так же модель изначально)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
ppo_trainer = PPOTrainer(config=config, model=model, ref_model=ref_model, tokenizer=tokenizer, dataset=dataset['train'], data_collator=collator)

In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

In [ ]:
sentiment_pipe = pipeline("sentiment-analysis", model="cointegrated/rubert-tiny-sentiment-balanced", device=device)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
text = "Это хреновая книга!"
sentiment_pipe(text, **sent_kwargs)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[[{'label': 'negative', 'score': 3.0925745964050293},
  {'label': 'neutral', 'score': -0.6827267408370972},
  {'label': 'positive', 'score': -2.1304874420166016}]]

In [ ]:
text = "Книга супер!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'negative', 'score': -3.954376459121704},
  {'label': 'neutral', 'score': -0.8630001544952393},
  {'label': 'positive', 'score': 4.4242095947265625}]]

In [ ]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0,
              "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

In [ ]:
output_min_length = 8
output_max_length = 32
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader),
                         total=dataset["train"].num_rows//ppo_trainer.dataloader.batch_sampler.batch_size):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[2]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

  0%|          | 0/13 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', 5000)

In [ ]:
#### get a batch from the dataset
bs = 32
game_data = dict()
dataset['train'].set_format("pandas")
df_batch = dataset['train'][:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]




/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
pd.DataFrame.from_dict(game_data)

,query,response (before),response (after),rewards (before),rewards (after)
0,Плохая идея.,"Допрежние дома, стоящие закрытые, принимая гостей по ошибке, все ведь живут там же, откуда и я. Публика, ведь","Я думал, ты скучаешь по Наше Дурочке). Само собой, решил она быть и моей. Спасибо ей",-0.002431,-1.259321
1,что лицемерного может быть,"то, чего ожидать от поднанимателей? И цена",у меня один))\n\nХм)\nей всё,2.487406,0.408705
2,Русскоязы,"чные слова: Робби (хьюмонда). Ingredo - персонал, популярный английский язык.\n\n","чное посольство) Мисс- Мисс))\nЯ тоже пришла\nмне сложно, но ПРАВИЛЬНО! классно, то",2.993591,-1.376948
3,"Массовая, та","составная часть самого крупного слова &mdash; characteros прочна, но одна из ее главных особенностей &md","самая очередь, которая протекала в 1993-й.... так сам день складывал) ) обожаю каждый день осить кухню и",0.823177,-0.252396
4,Нет в,"этом непосредственной проблемы, но не исключено, что и это вычислит поисковик. Церковная норма для этого - право совершать священнодействие вместе","занятиях так сложно завтракать))), мое счастье есть)\nмоему счастью всеха хорошо\nОпыт\nОх не всегда так радоеву",0.720534,-1.261084
5,Вы привели достаточно аргументов чтобы,заставить замолчать Анатолия как и можно более густота и горстку проблем всеми князьями на сегодня. &,посмотреть тебе)\nтебе понравится)\nУ меня любимый лапочка) всё правильно организовано) всё хорошо,0.037003,-0.908311
6,"Подождите, весь смысл","всей пьесы? Что же в действительности происходит?\n\nКОММЕНТАРИЙ: В том-то и дело, что любовь – понятие уз","послания был она.. she well, clint it needs her boot)\n\nю) странно, но я так сильно ей улыбнулась, что она",0.514185,-0.979280
7,И ничего,". Даже не верю в сыворотки? 4. Вывод -- бесплатная порция любая (я сам, если в энциклопедиях по питанию пишу","не знал)-)\nПросто рядом было всё чудесно.\nОтлично дома, все так хорошо!\nОна очень рада, ей понравился мой выбор.",0.889085,-1.325481
8,Ну прибористы обязаны проверять приборы,при расчёте расходов на свет и воду (заплатить что-то за бумагу))\nЗаданий за какой,"), ну 2 блогера )\nУ меня всё отлично\nа ты улыбаешься\nЯ рада, что получилось развиваться",1.734022,-0.395980
9,Да без обид,\nОльга Николаева\n\n\n9082727\tbelka-andree,)\nсмотря для чего это задание\nне знания мрут или лечатся)\nНе,1.401021,-0.784724


В целом, кажется, тексты стали немного позитивнее. По крайней мере, стало сильно больше улыбок в тексте